In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# Load data from CSV files
@st.cache
def load_data():
    initial_stock = pd.read_csv("initial_stock.csv")
    daily_demand = pd.read_csv("daily_demand.csv")
    shipment_plan = pd.read_csv("shipment_plan.csv")
    return initial_stock, daily_demand, shipment_plan

# Simulator logic
def calculate_safety_stock(demand_data, safety_days):
    """
    Calculate safety stock for each store, product, and day based on average past 7 days' sales.
    """
    safety_stock = demand_data.copy()
    safety_stock['Avg_Sales_7_Days'] = safety_stock.groupby(['Location', 'Product'])['Demand'].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )
    safety_stock['Safety_Stock'] = safety_stock['Avg_Sales_7_Days'] * safety_days
    return safety_stock[['Day', 'Location', 'Product', 'Safety_Stock']]

def simulate_fulfillment(initial_stock, daily_demand, safety_data, reservation_ratios):
    """
    Simulate the fulfillment rates based on safety stock and reservation ratios.
    """
    # Make a copy of stock data for simulation purposes
    stock_data = initial_stock.copy()
    stock_data.rename(columns={'Initial_Stock': 'Remaining_Stock'}, inplace=True)

    # Add columns for tracking fulfillment
    daily_demand['Fulfilled'] = 0  # How much demand is fulfilled
    daily_demand['Unfulfilled'] = 0  # How much demand is unfulfilled

    # Iterate through days
    for day in range(1, daily_demand['Day'].max() + 1):
        # Process demand for each store/platform and product
        for idx, row in daily_demand[daily_demand['Day'] == day].iterrows():
            location = row['Location']
            product = row['Product']
            demand = row['Demand']

            # Safety stock for this store, product, and day
            safety_stock_row = safety_data[
                (safety_data['Day'] == day) &
                (safety_data['Location'] == location) &
                (safety_data['Product'] == product)
            ]
            safety_stock = safety_stock_row['Safety_Stock'].iloc[0] if not safety_stock_row.empty else 0

            # Reservation ratio
            reservation_ratio = reservation_ratios.get((location, product), 1.0)

            # Effective stock available for fulfillment after reserving safety stock
            stock_row = stock_data[
                (stock_data['Location'] == location) &
                (stock_data['Product'] == product)
            ]
            remaining_stock = stock_row['Remaining_Stock'].iloc[0] if not stock_row.empty else 0
            shared_stock = max(remaining_stock - safety_stock, 0) * reservation_ratio

            # Fulfill as much demand as possible
            fulfilled = min(demand, shared_stock)
            unfulfilled = demand - fulfilled

            # Update stock and demand fulfillment
            daily_demand.loc[idx, 'Fulfilled'] = fulfilled
            daily_demand.loc[idx, 'Unfulfilled'] = unfulfilled
            stock_data.loc[
                (stock_data['Location'] == location) & (stock_data['Product'] == product),
                'Remaining_Stock'
            ] -= fulfilled

    return daily_demand

def calculate_fulfillment_rates(demand_data):
    """
    Calculate fulfillment rates by store and product.
    """
    fulfillment_rates = demand_data.groupby(['Location', 'Product']).agg(
        Total_Demand=('Demand', 'sum'),
        Total_Fulfilled=('Fulfilled', 'sum')
    )
    fulfillment_rates['Fulfillment_Rate'] = fulfillment_rates['Total_Fulfilled'] / fulfillment_rates['Total_Demand']
    return fulfillment_rates.reset_index()

# Set up the Streamlit webpage
st.title("Ship-from-Store Strategy Simulator")
st.sidebar.header("Simulation Parameters")

# Load data
initial_stock, daily_demand, shipment_plan = load_data()

# Get user inputs
safety_days = st.sidebar.slider("Safety Days", 0, 14, 7)  # Range: 0 to 14 days, default: 7 days
default_reservation_ratio = st.sidebar.slider("Default Reservation Ratio", 0.0, 1.0, 0.5)  # Default: 50%

# Reservation ratio adjustments for specific stores and products
reservation_ratios = {}
location_product_pairs = [(loc, prod) for loc in daily_demand['Location'].unique() for prod in daily_demand['Product'].unique()]
for location, product in location_product_pairs:
    reservation_ratios[(location, product)] = default_reservation_ratio

# Calculate safety stock
safety_stock = calculate_safety_stock(daily_demand, safety_days)

# Simulate fulfillment
simulated_demand = simulate_fulfillment(initial_stock, daily_demand, safety_stock, reservation_ratios)

# Calculate fulfillment rates
fulfillment_rates = calculate_fulfillment_rates(simulated_demand)

# Visualization
st.header("Fulfillment Rates by Store and Product")
fig = px.bar(
    fulfillment_rates,
    x='Location',
    y='Fulfillment_Rate',
    color='Product',
    barmode='group',
    title="Fulfillment Rates by Store and Product",
    labels={'Fulfillment_Rate': 'Fulfillment Rate'}
)
st.plotly_chart(fig)

st.header("Key Results")
st.write("Simulation Results Summary:")
st.write(fulfillment_rates)

st.header("Simulated Fulfillment Details (Daily)")
st.write("Detailed Fulfillment Data (Daily):")
st.dataframe(simulated_demand)